### IMPORTS

In [2]:
import numpy as np
import scipy as sp

$\textbf{QUESTION 1}$
$$
\textbf{Linear Program Objective Function:}\\
\text{min}\ 3x_1 + 2x_2
\\~\\
\textbf{Constraints:}\\
2x_1 + x_2 \ge 4\\
x_1 + x_2 \ge 3\\
x_1 + 2x_2 \le 10\\
x_1 \ge 0\\
x_2 \ge 0
\\~\\
\text{The feasible region is the shaded portion in the below graph.}
$$
<p align="center">
<img src="/home/leonardo_leads/Documents/SchoolDocs/ga_tech_masters/omscs_ml/fall2023/deterministic_optimization/lectures/week8Modules15and16/homework/images/Q1.png" width="600" height="600" />
</p>

$\textbf{QUESTION 2}$

$$
\textbf{Standard Form Linear Program:}\\
\textbf{Objective Function:}\\
\text{min}\ 3x_1 + 2x_2
\\~\\
\textbf{Constraints:}\\
2x_1 + x_2 - x_3 = 4\\
x_1 + x_2 - x_4 = 3\\
x_1 + 2x_2 +x_5 = 10\\
x_1, x_2, x_3, x_4, x_5 \ge 0
\\~\\
x = 
\begin{bmatrix}
x_1 & x_2 & x_3 & x_4 & x_5
\end{bmatrix}^T
\\~\\
c = 
\begin{bmatrix}
3 & 2 & 0 & 0 & 0
\end{bmatrix}^T
\\~\\
A =
\begin{bmatrix}
2 & 1 & -1 & 0 & 0\\
1 & 1 & 0 & -1 & 0\\
1 & 2 & 0 & 0 & 1
\end{bmatrix}
\\~\\
b = 
\begin{bmatrix}
4 & 3 & 10
\end{bmatrix}
$$

$\textbf{QUESTION 3}$
$$
\textbf{PART A}\\
$$

In [54]:
# c = np.array([3, 2, 0, 0, 0]).T
# A = np.matrix([[2, 1, -1, 0, 0],
#                [1, 1, 0, -1, 0],
#                [1, 2, 0, 0, 1],])
# b = np.array([4, 3, 10]).T

# Example c, A, x, b
c = np.array([-2, -3, 0, 0, 0]).T
A = np.matrix([[-1, 1, 1, 0, 0],
               [3, 2, 0, 1, 0],
               [2, 3, 0, 0, 1],])
x1, x2, x3, x4, x5 = 0, 0, 10, 60, 60
b = np.array([10, 60, 60]).T

# Indices of c, A, b
c_ind = [i for i in range(c.shape[0])]
A_col_ind = [i for i in range(A.shape[1])]
b_ind = [i for i in range(b.shape[0])]

# basis matrix is the BFS
basis = np.array([A[:, [2, 3, 4]]])
b_inv = np.linalg.inv(basis)
x_b = b_inv @ b
x_n = np.array([x1, x2]).T
c_b = c[[2, 3, 4]]
c_n = c[[0, 1]]
Aij = [A[:, 0], A[:, 1]]
k = 1

while True:

    # Check if BFS
    print(f"{k=}")
    
    if np.all(x_b >= 0):
        print(f"Current basic solution is a BFS: {x_b=}")

        # Generalize c_n[] and A1/A2
        c_n_bari = np.squeeze(np.asarray(c_n[0] - c_b.T @ b_inv @ Aij[0]))
        c_n_barj = np.squeeze(np.asarray(c_n[1] - c_b.T @ b_inv @ Aij[1]))
        print(f"{c_n_bari=}\n{c_n_barj=}")

        # Check if BFS is optimal solution
        if c_n_bari < 0 or c_n_barj < 0:            
            print(f"Current BFS is not optimal!")

            d_b = np.squeeze(np.asarray(-b_inv @ Aij[1]))
            d_n = np.array([0, 1])
            print(f"{d_b=}")
            
            # Check if optimal solutions are unbounded
            if np.any(d_b < 0):
                print("Do not have an unbounded optimal solution.")

                # Minimum-Ratio Test
                theta = np.min(x_b / -d_b)
                theta_b, theta_n = np.squeeze(np.asarray(x_b / -d_b)), np.array([0, theta])
                print(f"{theta_b=}\n{theta_n=}")

                # Bland's Rule
                '''
                Choose the entering nonbasic variable x_j s.t. j is the smallest index with c_j^bar < 0
                Choose the exiting basic variable x_i s.t. i is the smallest index that achieves the min ratio
                '''
                c_n_bar = [c_n_bar1, c_n_bar2]
                entering_var = min(c_n_bar)
                entering_var_index = c_n_bar.index(entering_var)
                exiting_var = min(theta_b)
                exiting_var_index = np.where(theta_b == exiting_var)[0][0]
                print(f"{entering_var_index=}\n{exiting_var_index=}")
                
                # New data: NEEDS TO BE GENERALIZED
                basis[0][:, exiting_var_index] = np.squeeze(np.asarray(A[:, entering_var_index]))
                print(f"{'&'*10}\n{basis=}\n{'&'*10}")
                b_inv = np.linalg.inv(basis)
                c_b[exiting_var_index] = c[entering_var_index]
                c_n[entering_var_index] = c[exiting_var_index + 2]
                Aij[entering_var_index] = A[:, exiting_var_index + 2]
                x_b = b_inv @ b
                print(f"{x_b=}")

            else:
                print("Do have unbounded optimal solution.")
                break
            
        else:
            print(f"Current BFS is optimal!")
            print(f"{x_b=}")
            break

    else:
        print("Current basic solution is not a BFS")
        break

    k += 1


k=1
Current basic solution is a BFS: x_b=array([[10., 60., 60.]])
c_n_bari=array(-2.)
c_n_barj=array(-3.)
Current BFS is not optimal!
d_b=array([-1., -2., -3.])
Do not have an unbounded optimal solution.
theta_b=array([10., 30., 20.])
theta_n=array([ 0., 10.])
entering_var_index=1
exiting_var_index=0
&&&&&&&&&&
basis=array([[[1, 0, 0],
        [2, 1, 0],
        [3, 0, 1]]])
&&&&&&&&&&
x_b=array([[10., 40., 30.]])
k=2
Current basic solution is a BFS: x_b=array([[10., 40., 30.]])
c_n_bari=array(-5.)
c_n_barj=array(3.)
Current BFS is not optimal!
d_b=array([-1.,  2.,  3.])
Do not have an unbounded optimal solution.
theta_b=array([ 10., -20., -10.])
theta_n=array([  0., -20.])
entering_var_index=1
exiting_var_index=1
&&&&&&&&&&
basis=array([[[1, 1, 0],
        [2, 2, 0],
        [3, 3, 1]]])
&&&&&&&&&&


LinAlgError: Singular matrix

Test Simplex

Test Simplex

Test Simplex

In [4]:
def Simplex(A, b, c):
    '''Takes input vars, computs corresponding values,
    then uses while loop to iterate until a basic optimal solution is reached.
    RETURNS: basic_cost_coeff, basic_cost_index, non_basic_cost, nonbasic_cost_index, bHat, cnHat.
    basic_cost_coeff, cbIndex is final basic variable values, and indices
    non_basic_cost, cnIndex is final nonbasic variable values and indices
    bHat is final solution values, 
    cnHat is optimality condition'''
    
    # sizes of basic and nonbasic vectors
    basicSize = A.shape[0]
    nonbasicSize = A.shape[1] - basicSize # n - m, number of variables
    print(f"{basicSize=}")
    print(f"{nonbasicSize=}")
        
    # global index tracker of variables of basic and nonbasic variables (objective)
    cindx = [i for i in range(0, len(c))]
    print(f"{cindx=}")
    
    # basic variable coefficients
    basic_cost_coeff = np.array(c[:basicSize])
    print(f"{basic_cost_coeff=}")
        
    # nonbasic variable coefficients
    non_basic_cost = np.array(c[basicSize:])
    print(f"{non_basic_cost=}")

    start_AInd = np.array([2, 3, 4])
    basis = A[:, start_AInd]
    basic_var = np.linalg.inv(basis)
    print(f"1 {basis=}")
    k = 1
 
    # run core simplex method until reach the optimal solution
    while True:
        if k == 3:
            break
        
        basic_cost_index = cindx[:basicSize]
        nonbasic_cost_index = cindx[basicSize:]
    

        
        # nonbasic variable matrix
        N = A[:, nonbasic_cost_index]

        # bHat, the values of the basic variables
        # recall that at the start the basic variables are the slack variables, and 
        # have values equal to the vector b (as primary variables are set to 0 at the start)
        bHat = basic_var @ b
        yT = basic_cost_coeff @ basic_var
        
        # use to check for optimality, determine variable to enter basis
        cnHat = non_basic_cost - (yT @ N)
        
        
        # find indx of minimum value of cnhat, this is the variable to enter the basis
        cnMinIndx = np.argmin(cnHat)

        # break out of loop, returning values if all values of cnhat are above 0
        if all(i >= 0 for i in list([cnHat[0,0], cnHat[0,1]])):
            # use basic_cost_index to get index values of variables in bHat, and the corresponding index
            # values in bHat are the final solution values for each of the corresponding variables
            # ie value 0 in dbIndx corresponds with first variable, so whatever the index for the 0 is
            # is the index in bHat that has the solution value for that variable.
            return f"{k=}\n{basic_cost_coeff=}\n{basic_cost_index=}\n{non_basic_cost=}\n{nonbasic_cost_index=}\n{bHat=}\n{cnHat=}"
    
        # this is the index for the column of coeffs in A for the given variable
        indx = cindx[cnMinIndx]

        Ahat = basic_var @ A[:, indx]
        
        # now we want to iterate through Ahat and bHat and pick the minimum ratios
        # only take ratios of variables with Ahat_i values greater than 0
        # pick smallest ratio to get variable that will become nonbasic.
        ratios = []
        for i in range(0, len(bHat)):
            Aval = Ahat[i]
            Bval = bHat[i]

            # don't look at ratios with val less then or eqaul to 0, append to keep index
            if(Aval <= 0):
                ratios.append(10000000)
                continue
            ratios.append(Bval / Aval)

        ratioMinIndx = np.argmin(ratios)

        # switch basic and nonbasic variables using the indices.
        non_basic_cost[cnMinIndx], basic_cost_coeff[ratioMinIndx] = basic_cost_coeff[ratioMinIndx], non_basic_cost[cnMinIndx]
        
        # switch global index tracker indices
        cindx[cnMinIndx], cindx[ratioMinIndx + nonbasicSize] = cindx[ratioMinIndx + nonbasicSize], cindx[cnMinIndx]
        # basis matrix
        basis = A[:, basic_cost_index]
        print(f"2 {basis=}")
        basic_var = np.linalg.inv(basis)
        k += 1

c = np.array([-2, -3, 0, 0, 0])
A = np.matrix([[-1, 1, 1, 0, 0],
               [3, 2, 0, 1, 0],
               [2, 3, 0, 0, 1],])
b = np.array([10, 60, 60])
print(Simplex(A, b, c))

basicSize=3
nonbasicSize=2
cindx=[0, 1, 2, 3, 4]
basic_cost_coeff=array([-2, -3,  0])
non_basic_cost=array([0, 0])
1 basis=matrix([[1, 0, 0],
        [0, 1, 0],
        [0, 0, 1]])
k=1
basic_cost_coeff=array([-2, -3,  0])
basic_cost_index=[0, 1, 2]
non_basic_cost=array([0, 0])
nonbasic_cost_index=[3, 4]
bHat=matrix([[10., 60., 60.]])
cnHat=matrix([[3., 0.]])


In [5]:
#print(sp.optimize.linprog(A_eq=A, b_eq=b, c=c, method='simplex', options={'bland': True}))